# Sign Connect Model | Real Time Sign Language Translation App
The format used for annotations is Pascal VOC XML.
To run the notebook, data should be downloaded from [github.com/Safayy/SignConnectMobile/data](https://). All files should be placed in your Google Drive, under a directory named SignConnect.

### Setup TensorFlow Object Detection

In [ ]:
!pip install tensorflow==2.15.0

!git clone https://github.com/tensorflow/models.git
%cd models/research
!sed -i 's/tf-models-official>=2.5.1/tf-models-official==2.15.0/g' object_detection/packages/tf2/setup.py #Fix setup.py bug to allow install of different TensorFlow version
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

### Import Libraries

In [24]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf

### Get Files from Drive

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [ ]:
%cd /content
!unzip /mydrive/SignConnectModel/images.zip -d .
!unzip /mydrive/SignConnectModel/annotations.zip -d .

### Split Training and Testing Dataset


In [29]:
!mkdir test_labels train_labels
!ls annotations/* | sort -R | head -40 | xargs -I{} mv {} test_labels/
!ls annotations/* | xargs -I{} mv {} train_labels/

mkdir: cannot create directory ‘test_labels’: File exists
mkdir: cannot create directory ‘train_labels’: File exists


### Create Label Files

In [30]:
labels = [
    {'name':'letter_A', 'id':1},
    {'name':'letter_B', 'id':2},
    {'name':'letter_C', 'id':3},
    {'name':'letter_D', 'id':4},
    {'name':'letter_E', 'id':5},
    {'name':'letter_F', 'id':6},
    {'name':'letter_G', 'id':7},
    {'name':'letter_H', 'id':8},
    {'name':'letter_I', 'id':9},
    {'name':'letter_J', 'id':10},
    {'name':'letter_K', 'id':11},
    {'name':'letter_L', 'id':12},
    {'name':'letter_M', 'id':13},
    {'name':'letter_N', 'id':14},
    {'name':'letter_O', 'id':15},
    {'name':'letter_P', 'id':16},
    {'name':'letter_Q', 'id':17},
    {'name':'letter_R', 'id':18},
    {'name':'letter_S', 'id':19},
    {'name':'letter_T', 'id':20},
    {'name':'letter_U', 'id':21},
    {'name':'letter_V', 'id':22},
    {'name':'letter_W', 'id':23},
    {'name':'letter_X', 'id':24},
    {'name':'letter_Y', 'id':25}
]
!touch label_map.pbtxt
with open('label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

!touch label.txt
with open('label.txt', 'w') as label_file:
    for label in labels:
        label_file.write(label['name'] + '\n')

### Create CSV Labels

In [31]:
# Convert files to CSV using RDB converter
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name)
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


### Generate record files

In [32]:
!cp /mydrive/SignConnectModel/data/generate_tfrecord.py .

!python generate_tfrecord.py train_labels.csv  label_map.pbtxt images/ train.record
!python generate_tfrecord.py test_labels.csv  label_map.pbtxt images/ test.record

2024-05-06 09:47:46.835746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 09:47:46.835854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 09:47:46.856734: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 09:47:52.567688: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
groups:  21% 80/383 [00:12<00:45,  6.65it/s]
Traceback (most recent call last):
  File "/content/generate_tfrecord.py", line 137, in <module>
    tf_example = create_tf_example(group, path, class_dict)
  File "/content/generate_tfrecord.py",

## Train Model

In [14]:
# Alternatively use previously trained model. Skip remainder of this section.
# !unzip /mydrive/SignConnectModel/training training

In [15]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2024-05-06 08:16:11--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.193.207, 172.217.204.207, 172.217.203.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.193.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  --.-KB/s    in 0.1s    

2024-05-06 08:16:11 (189 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320

In [16]:
# Get pipeline.config from Drive. File was modified manually
!cp /mydrive/SignConnectModel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config .

In [18]:
# Local fix for tensorflow 2.15.0 bug
!sed -i 's/_ops\.cond/_ops_cond\.cond/g; s/flow_ops\.case/flow_case\.case/g' /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py
!sed -i '45i from tensorflow.python.ops import cond as control_flow_ops_cond\nfrom tensorflow.python.ops import control_flow_case\n' /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py

In [22]:
#!ls /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
!python models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config \
  --model_dir=training \
  --alsologtostderr

2024-05-06 08:31:47.597206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 08:31:47.597274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 08:31:47.599797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 08:31:49.590867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0506 08:31:53.509994 140567438409728 mirrored_strategy.py:423] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/devic

### Save Trained Model to Drive

In [ ]:
!zip -r training training

## Export graph

In [21]:
!python /content/models/research/object_detection/exporter_main_v2.py \
  --trained_checkpoint_dir=training \
  --pipeline_config_path=ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config \
  --output_directory inference_graph

2024-05-06 08:21:02.880458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 08:21:02.880516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 08:21:02.888196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 08:21:05.135758: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0506

## Export TFLite Graph

In [ ]:
!cp /content/gdrive/MyDrive/SignConnectModel/tfexample_decoder.py /usr/local/lib/python3.10/dist-packages/tf_slim/data/.

In [ ]:
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config \
  --trained_checkpoint_dir training \
  --output_directory tflite

2024-05-04 10:56:57.076766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 10:56:57.076835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 10:56:57.078760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-04 10:56:59.065669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0504 10:57:07.002274 138237796151296 api.py:460] feature_map_spatial_dims: [(40, 40), (20, 20), (10, 10), (5, 5), (3, 3)]
I0504 10:57:13.578443 138237796151296 api.py:460] feature_map_spatial_dims: [(40, 40), (20, 20), (10, 10), (5, 5), (3,

## Convert to TFLite

In [ ]:
!tflite_convert \
  --saved_model_dir=tflite/saved_model \
  --output_file=tflite/model.tflite

2024-05-04 12:28:39.766379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 12:28:39.766454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 12:28:39.767856: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 180, Total Ops 340, % non-converted = 52.94 %
 * 180 ARITH ops

- arith.constant:  180 occurrences  (f32: 174, i32: 6)



  (f32: 12)
  (f32: 2)
  (f32: 72)
  (f32: 1)
  (f32: 51)
  (f32: 1)
  (f32: 4)
  (f32: 14)


## Attach Metadata

In [ ]:
!pip install tflite-support

In [ ]:
!mkdir tflite/tflite_with_metadata

In [ ]:
# Attach Metadata to TFLite
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

import flatbuffers
import os
from tensorflow_lite_support.metadata import metadata_schema_py_generated as _metadata_fb
from tensorflow_lite_support.metadata.python import metadata as metadata
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_info
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_writer
from tensorflow_lite_support.metadata.python.metadata_writers import writer_utils

ObjectDetectorWriter = object_detector.MetadataWriter

_MODEL_PATH = "tflite/model.tflite"
_LABEL_FILE = "label.txt"
_SAVE_TO_PATH = "tflite/tflite_with_metadata/model.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "SSD_Detector"
model_meta.description = (
    "Identify which of a known set of objects might be present and provide "
    "information about their positions within the given image or a video "
    "stream.")

# Create input info
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Create output info
output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.BoundingBoxProperties)
output_location_meta.content.contentProperties = (
    _metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
output_location_meta.content.contentProperties.type = (
    _metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (
    _metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2

output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "category"
output_class_meta.description = "The categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("labelmap.txt")
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
output_class_meta.associatedFiles = [label_file]

output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2

output_number_meta = _metadata_fb.TensorMetadataT()
output_number_meta.name = "number of detections"
output_number_meta.description = "The number of the detected boxes."
output_number_meta.content = _metadata_fb.ContentT()
output_number_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_number_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())

# Create subgraph info
group = _metadata_fb.TensorGroupT()
group.name = "detection result"
group.tensorNames = [
    output_location_meta.name, output_class_meta.name,
    output_score_meta.name
]
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]

subgraph.outputTensorMetadata = [
    output_location_meta, output_class_meta, output_score_meta,
    output_number_meta
]
subgraph.outputTensorGroups = [group]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

Metadata populated:
{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
    